In [1]:
#Select VASCA sources based on chosen criteria

In [2]:
region_name = "ALL_10-800_LOOSE" #"CAINGSGII_10-800" #"TDS" #"CAINGSGII_10-800" #"WD" #"MDIS_10-800" #"TDS" #  _ELAISN1
region_cat_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"

In [3]:
### Prepare tt_sources with variables for selection

In [4]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from vasca.region import Region
import vasca.visualization as vvis
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, get_col_cycler,add_ogrp, color_palette, nb_fig
from astropy import units as uu
from astropy.table import Column

# Get region
rc = Region()
rc.load_from_fits(region_cat_fname)
rc.tt_sources.sort(["dec"])

#Add Object group to source table
rc.copy_table_columns("tt_sources","tt_simbad",["otype","ogrp","match_distance","sp_type","main_id"],"rg_src_id")
rc.tt_sources.rename_column("match_distance","match_distance_simbad")
rc.tt_sources.rename_column("ogrp","ogrp_simbad")
sel_simbad = rc.tt_sources["sel"].data
print(f"Number of SIMBAD matches is: {sel_simbad.sum()}  ({100* sel_simbad.sum() / len(rc.tt_sources)} )%")

rc.copy_table_columns("tt_sources","tt_gaiadr3",["PQSO", "PGal", "PSS","match_distance","Gmag_abs","BP-RP", "ogrp", "Plx","e_Plx","Plx_dist"],"rg_src_id")
rc.tt_sources.rename_column("match_distance","match_distance_gaiadr3")
rc.tt_sources.rename_column("ogrp","ogrp_gaiadr3")
sel_gaiadr3 = rc.tt_sources["sel"].data
print(f"Number of GAIA matches is: {sel_gaiadr3.sum()}  ({100*sel_gaiadr3.sum() / len(rc.tt_sources)} )%")

sel_match = np.array(sel_simbad+sel_gaiadr3, dtype=bool)
print(f"Total number of matches is: {sel_match.sum()}  ({100*sel_match.sum() / len(rc.tt_sources)} )%")

#Copy lomb scargle result into tt_sources
rc.copy_table_columns("tt_sources","tt_lombscargle",["ls_peak_pval", "ls_model_rchiq"],"rg_src_id")

display(rc.tt_simbad[rc.tt_simbad["ogrp"]=="WD*"])

Number of SIMBAD matches is: 5508  (52.89541918755402 )%
Number of GAIA matches is: 7952  (76.36608086046289 )%
Total number of matches is: 8696  (83.51099587054644 )%


main_id,ra,dec,ra_prec,dec_prec,coo_err_maja,coo_err_mina,coo_err_angle,coo_qual,coo_wavelength,coo_bibcode,otype,otypes,distance_distance,distance_q,distance_unit,distance_merr,distance_perr,distance_method,distance_bibcode,match_distance,rvz_type,rvz_radvel,rvz_error,rvz_qual,rvz_wavelength,rvz_bibcode,z_value,sp_type,sp_qual,sp_bibcode,script_number_id,rg_src_id,simbad_match_id,ogrp
,deg,deg,,,mas,mas,deg,,,,,,,,,,,,,arcsec,,km / s,km / s,,,,,,,,,,,
bytes32,float64,float64,int16,int16,float32,float32,int16,bytes1,bytes1,bytes32,bytes32,bytes32,float64,bytes1,bytes4,float64,float64,bytes8,bytes19,float32,bytes1,float64,float32,bytes1,bytes1,bytes32,float64,bytes32,bytes32,bytes32,int32,int32,int32,bytes8
GALEX J221729.2+000833,334.3718966666666,0.14262111111111112,14,14,0.197,0.177,90,A,O,2020yCat.1350....0G,WD*,WD*|WD*|WD*|WD*|WD*|WD*|*|G|Opt|,319.8670,--,pc,-23.3176,23.3176,paral,2020yCat.1350....0G,0.4493,z,-0.000,0.000,E,--,2008ApJS..175..297A,-0.0000100,DB,D,2006ApJS..167...40E,101,4993,77,WD*
GALEX J221613.5+010456,334.05641333333324,1.0822822222222221,14,14,0.290,0.205,90,A,O,2020yCat.1350....0G,WD*,WD*|WD*|WD*|WD*|WD*|*|Opt|UV|WD?,153.2380,--,pc,-8.2609,8.2609,paral,2020yCat.1350....0G,0.1086,z,0.000,0.000,E,--,2008ApJS..175..297A,0.0000900,DA,D,2006ApJS..167...40E,315,14504,234,WD*
VIPERS 406014492,334.8938645833333,0.881735,14,14,0.349,0.319,90,A,O,2020yCat.1350....0G,WD?,*|*|G|WD?|WD?,--,--,--,--,--,--,--,0.4710,--,--,--,--,--,--,--,--,--,--,525,30475,380,WD*
V* V2276 Sgr,306.59244125,-43.6754825,14,14,0.331,0.248,90,A,O,2020yCat.1350....0G,CV*,CV*|V*|*|WD?,--,--,--,--,--,--,--,0.2714,--,--,--,--,--,--,--,--,--,--,585,34280,419,WD*
V* IY UMa,160.98645291666665,58.12553333333334,14,14,0.036,0.046,90,A,O,2020yCat.1350....0G,CV*,PM*|CV*|CV*|V*|PM*|NIR|*|*|CV?|C,182.0470,--,pc,-2.2337,2.2337,paral,2020yCat.1350....0G,0.3422,z,0.001,0.000,E,--,2009ApJS..182..543A,0.0005300,--,--,--,794,50388,539,WD*
GALEX J100210.9+025821,150.54527416666664,2.9726219444444446,14,14,0.219,0.200,90,A,O,2020yCat.1350....0G,WD*,WD*|WD*|WD*|WD*|WD*|WD*|*|Opt|UV,526.9200,--,pc,-52.0300,52.0300,ST-L,2017MNRAS.469.2102A,0.5164,z,0.000,0.000,E,--,2008ApJS..175..297A,0.0000600,DA,D,2006ApJS..167...40E,846,52994,580,WD*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GALEX J191654.1-244628,289.2254916666666,-24.774540555555554,14,14,0.153,0.150,90,A,O,2020yCat.1350....0G,WD?,*|UV|WD?|WD?|WD?,162.8480,--,pc,-5.1050,5.1050,paral,2020yCat.1350....0G,0.6672,--,--,--,--,--,--,--,--,--,--,7488,1070938,4098,WD*


### Print info for selected sources

In [5]:
# Make selection
sel_otype = rc.tt_sources["otype"] ==  "RR*"# "No*"#"HS?" #"BS*" #"PM*" # # "SN*" #"EB*"
sel_otypes = (rc.tt_sources["otype"] ==  "WD?")  + (rc.tt_sources["otype"] =="WD*") + (rc.tt_sources["ogrp_gaiadr3"] =="WD*")
sel_flux_nxv = rc.tt_sources["flux_nxv"][:,0]>2
sel_flux_hr = rc.tt_sources["hr"]>2
sel_src_id = rc.tt_sources["rg_src_id"]==15179
sel_period = (rc.tt_sources["ls_peak_pval"]>-0.5) * (rc.tt_sources["ls_peak_pval"]<0.000000573303) * (rc.tt_sources["ls_model_rchiq"]<1.1)# # 3 sigma 0.002699796063, 4 sigma 0.000063342484, 5 sigma 0.000000573303
sel_srcs =  sel_otypes #sel_period #sel_otypes  #sel_src_id  #sel_flux_nxv #sel_otype # sel_flux_nxv_fuv # sel_flux_hr#

#Print oout list of source IDs to be used for further use, e.g. in inspect_sources.ipynb
print("\nsrcs_ids = [", end="")
for src_id in rc.tt_sources[sel_srcs]["rg_src_id"].data:
    print(src_id,",", end="")
print("]\n")
print("Total Nr. of sources:",sel_srcs.sum(),"\n")

#Display match and simbad table for selected sources
cols_write = ["ra","dec","ogrp_gaiadr3","otype","sp_type","main_id","nr_det","flux","flux_cpval","flux_ne","Plx_dist","ls_peak_pval","ls_model_rchiq","rg_src_id"]
rc.tt_sources[sel_srcs][cols_write].pprint_all() #["sp_type","main_id"]
rc.tt_sources[sel_srcs][cols_write].write("sources.ecsv",overwrite=True,delimiter=',')


srcs_ids = [253670 ,253214 ,540235 ,1037284 ,659787 ,680119 ,256433 ,1839206 ,498065 ,666224 ,1070938 ,1069077 ,714140 ,236839 ,764827 ,3074917 ,13580 ,645730 ,639720 ,645215 ,162697 ,221349 ,4993 ,228958 ,30475 ,14504 ,120265 ,138783 ,139480 ,52994 ,1805505 ,2527679 ,381336 ,532554 ,533185 ,204419 ,390864 ,525310 ,331466 ,489430 ,340464 ,673175 ,79477 ,691408 ,583838 ,50388 ,261213 ,]

Total Nr. of sources: 47 

        ra                 dec          ogrp_gaiadr3 otype sp_type           main_id              nr_det            flux                    flux_cpval                    flux_ne            Plx_dist  ls_peak_pval  ls_model_rchiq rg_src_id
       deg                 deg                                                                                    1e-06 Jy                                                                      pc                                           
------------------ -------------------- ------------ ----- ------- ---------------------------- --------- 